## SETI end-to-end narrowband drifting tone detection over multiple raw files
This notebook sequences through multiple raw files and looks for drifting narrowband tones.  

This is a general-purpose notebook for real observations such as Voyager or commensal pointings.

For each test file, notebook "01_seti_end_to_end.ipynb" is called which performs the following:
1) Inputs each RAW file 
2) Runs Rawspec to generate a filterbank .h5 spectrogram file with specified fine FFT size and integration factor n_sti
3) Runs TurboSETI and/or seticore and compiles a list of detections and compute time (wall clock)

Note that these sets of files can be done with various branches of seticore or seticore2 and detection SNR or timing comparisons
can be made between branches over multiple runs of this notebook.  For each run of this notebook, 
the desired seticore branch needs to be checked out and compiled, and 
the string "test_case" needs to be set in 01_seti_end_to_end.ipynb to indicate the branch.

The raw files are copied locally from bulk storage if needed, or rawspec can be run directly from bulk storage without copying locally. 
If local space is limited, raw files can be deleted after each run of this notebook. 

Plots are generated in each "01_seti_end_to_end.ipynb" call.

In [ ]:
%load_ext autoreload
%autoreload 2

parameters_are_undefined = False     # use parameter defaults settings in 01_seti_end_to_end.ipynb

app_to_run = 'seticore-20'
# app_to_run = 'bliss-20'

delete_raw_file_after_use = True
display_figs01 = False
plot_all_dets = True
plot_multichirp_dets1 = True
plot_multichirp_dets2 = False
verbose = True
plot_sg_psd_zoom = True
plot_psd_zoom = False
plot_sg_zoom = False
plot_sg_wb = True
plot_psd_wb = True
zoom_bw_MHz = .012  # +/- 6 KHz
max_n_zoom_plots = 50

run_turbo = False
fb_ext = '.h5'

search_max_drift = 10    # Hz/sec
search_min_drift = -10   # ignored except for plotting

delete_raw_file_enable = True   # =True will delete raw file from $RAWDIR, if not present initially in $RAWDIR,
                                # and copied from $RAW_BACKUP_BASE_DIR/raw_backup_subdir
                                # =False will keep raw file in $RAWDIR regardless
delete_h5_sg_file = True        # =True h5 sg file will be deleted after finishing (reduce file space in $SGDIR)

enable_raw_backup_copy = False  # =True copy to $RAWDIR if needed from $RAW_BACKUP_BASE_DIR/raw_backup_subdir
                                # =False rawspec will read directly from $RAW_BACKUP_BASE_DIR/raw_backup_subdir without file copy

# settings for GBT, override otherwise
fine_fft_size = 1024*1024
n_sti = 4
# n_sti = 32

if (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015'] # 23.7 dB, -.504 Hz/sec
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc3_guppi_57386_VOYAGER1_0004'] 
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (1):
    raw_backup_subdir = 'raw_multichirp/'
    delete_raw_file_enable = False
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec']
        # 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec',
        # 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec']
    fig_dir = './plots/'
    n_sti = 16
    plot_sg_wb = False
    plot_psd_wb = False
    plot_sg_psd_zoom = False
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_80036_DIAG_VOYAGER-1_0011', # 24.4 dB, -.279 Hz/sec
                        'blc23_guppi_59046_80354_DIAG_VOYAGER-1_0012', # no det for 0000
                        'blc23_guppi_59046_80672_DIAG_VOYAGER-1_0013', # 20.93 dB, -.52 Hz/sec
                        'blc23_guppi_59046_80989_DIAG_VOYAGER-1_0014', # no det for 0000
                        'blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015', # 26 dB, -.44 Hz/sec Up to 14 files!
                        'blc23_guppi_59046_81628_DIAG_VOYAGER-1_0016'] # no det for 0000
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
elif (1):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0010',
                          'blc2_guppi_57388_HIP113357_0010',
                          'blc3_guppi_57388_HIP113357_0010',
                          'blc4_guppi_57388_HIP113357_0010',
                          'blc5_guppi_57388_HIP113357_0010',
                          'blc6_guppi_57388_HIP113357_0010'] 
    fig_dir = './raw_gbt_57388_HIP/'
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (0):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0012',
                          'blc2_guppi_57388_HIP113357_0012',
                          'blc3_guppi_57388_HIP113357_0012',
                          'blc4_guppi_57388_HIP113357_0012',
                          'blc5_guppi_57388_HIP113357_0012',
                          'blc6_guppi_57388_HIP113357_0012'] 
    fig_dir = './raw_gbt_57388_HIP/'
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (0):
    raw_backup_subdir = 'raw_gbt_59103_HIP/'
    raw_file_stem_list = ['blc40_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc41_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc42_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc43_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc44_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc45_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc46_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc47_guppi_59103_03394_DIAG_HIP95631_0015'] 
    fig_dir = './raw_gbt_59103_HIP/'
elif (0):
    raw_backup_subdir = 'raw_gbt_59103_Kepler/'
    raw_file_stem_list = ['blc40_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc41_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc42_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc43_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc44_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc45_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc46_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc47_guppi_59103_01984_DIAG_KEPLER-160_0010'] 
    fig_dir = './raw_gbt_59103_Kepler/'
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_TIC/'
    raw_file_stem_list = ['blc40_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc41_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc42_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc43_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc44_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc45_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc46_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc47_guppi_59103_03721_DIAG_TIC159107668_0016'] 
    fig_dir = './raw_gbt_59103_TIC/'

n_raw_file = len(raw_file_stem_list)

print('raw_file_stem_list=')
for i_raw in range(n_raw_file):
    print(f'{raw_file_stem_list[i_raw]}')
             
print(f'Figures will appear in in {fig_dir}')

raw_file_stem_list=
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec
Figures will appear in in ./plots/


#### Run end-to-end test over parameter sweeps

In [2]:

for i_raw in range(n_raw_file):

    raw_file_stem = raw_file_stem_list[i_raw]
    print(f'\n\nRaw file {i_raw+1} of {n_raw_file}: {raw_file_stem}\n\n')

    try:      
        %run ./01_seti_end_to_end.ipynb
    except:
        print(f'\n\nError Occurred for Raw file {raw_file_stem}, skip\n\n')

    if delete_raw_file_after_use & (not local_raw_file_found):
        # delete raw file to limit local file storage
        try:
            print('Deleting ',raw_file_delete_spec)
            os.system('rm '+raw_file_delete_spec)
        except:
            print(' ')

            



Raw file 1 of 3: gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec


search_app_name='seticore2', search_app_string='Seticore', branch='sc2', test_case='sc2-10', search_z_threshold=10
sigma_drift = [0.75 1.5  3.  ]
/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw
Raw file found in raw directory
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec
['gbt', 'chirp80', '0.0040V', '6002.20', '6003.80', 'm10.0', '10.0', 'Hzsec', '22.91sec']
n_chirp = 80 f_start1_MHz = 6002.2 f_stop1_MHz = 6003.8 sig_min_drift = -10.0 sig_max_drift = 10.0
signal_level = 0.004, n_chirp = 80
f_start1_MHz = 6002.2, f_stop1_MHz = 6003.8,  f_start_incr = 20253.164556503296, df_dt_incr = 0.2531645569620249
f_start_truth[0] = 6002200000.0 f_start_truth[-1] = 6003800000.0
drift_rate_truth[0] = -10.0 drift_rate_truth[-1] = 10.0
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0

dedoppler elapsed time: 1s



Seticore search: 0.78 sec 0.19 sec/coarse channel
E(SNR)    [22.48 21.31 19.39] dB
1 SNR 13.98 dB, 6002.200 MHz, -8.945 Hz/sec
2 SNR 14.30 dB, 6002.220 MHz, -7.481 Hz/sec
3 SNR 14.15 dB, 6002.240 MHz, -8.619 Hz/sec
4 SNR 14.51 dB, 6002.261 MHz, -9.270 Hz/sec
5 SNR 14.50 dB, 6002.281 MHz, -7.318 Hz/sec
6 SNR 14.79 dB, 6002.301 MHz, -9.270 Hz/sec
7 SNR 14.39 dB, 6002.321 MHz, -6.505 Hz/sec
8 SNR 14.27 dB, 6002.342 MHz, -7.969 Hz/sec
9 SNR 14.18 dB, 6002.362 MHz, -6.993 Hz/sec
10 SNR 14.92 dB, 6002.382 MHz, -6.668 Hz/sec
11 SNR 14.56 dB, 6002.403 MHz, -6.180 Hz/sec
12 SNR 15.15 dB, 6002.423 MHz, -8.619 Hz/sec
13 SNR 15.21 dB, 6002.443 MHz, -6.343 Hz/sec
14 SNR 15.01 dB, 6002.463 MHz, -6.668 Hz/sec
15 SNR 15.52 dB, 6002.484 MHz, -5.692 Hz/sec
16 SNR 15.41 dB, 6002.504 MHz, -4.716 Hz/sec
17 SNR 15.37 dB, 6002.524 MHz, -5.042 Hz/sec
18 SNR 15.88 dB, 6002.544 MHz, -5.529 Hz/sec
19 SNR 15.71 dB, 6002.565 MHz, -5.367 Hz/sec
20 SNR 15.88 dB, 6002.585 MHz, -6.017 Hz/sec
21 SNR 16.19 dB, 6002.605

dedoppler elapsed time: 1s



Seticore search: 0.97 sec 0.24 sec/coarse channel
E(SNR)    [23.76 22.77 20.99] dB
1 SNR 15.19 dB, 6002.200 MHz, -8.852 Hz/sec
2 SNR 15.43 dB, 6002.220 MHz, -9.758 Hz/sec
3 SNR 14.93 dB, 6002.240 MHz, -9.479 Hz/sec
4 SNR 15.77 dB, 6002.261 MHz, -9.479 Hz/sec
5 SNR 15.12 dB, 6002.281 MHz, -8.712 Hz/sec
6 SNR 15.57 dB, 6002.301 MHz, -9.270 Hz/sec
7 SNR 15.17 dB, 6002.321 MHz, -8.643 Hz/sec
8 SNR 15.63 dB, 6002.342 MHz, -8.224 Hz/sec
9 SNR 15.70 dB, 6002.362 MHz, -8.434 Hz/sec
10 SNR 15.76 dB, 6002.382 MHz, -7.667 Hz/sec
11 SNR 15.56 dB, 6002.403 MHz, -7.946 Hz/sec
12 SNR 16.17 dB, 6002.423 MHz, -7.667 Hz/sec
13 SNR 16.12 dB, 6002.443 MHz, -7.388 Hz/sec
14 SNR 16.09 dB, 6002.463 MHz, -6.691 Hz/sec
15 SNR 16.61 dB, 6002.484 MHz, -5.924 Hz/sec
16 SNR 16.47 dB, 6002.504 MHz, -6.412 Hz/sec
17 SNR 16.60 dB, 6002.524 MHz, -6.203 Hz/sec
18 SNR 17.06 dB, 6002.544 MHz, -5.297 Hz/sec
19 SNR 16.91 dB, 6002.565 MHz, -6.133 Hz/sec
20 SNR 16.94 dB, 6002.585 MHz, -5.785 Hz/sec
21 SNR 17.46 dB, 6002.605

dedoppler elapsed time: 1s



Seticore search: 1.26 sec 0.31 sec/coarse channel
E(SNR)    [25.66 24.2  22.21] dB
1 SNR 16.14 dB, 6002.200 MHz, -9.986 Hz/sec
2 SNR 16.89 dB, 6002.220 MHz, -9.790 Hz/sec
3 SNR 16.26 dB, 6002.240 MHz, -9.335 Hz/sec
4 SNR 17.13 dB, 6002.261 MHz, -9.400 Hz/sec
5 SNR 16.33 dB, 6002.281 MHz, -8.587 Hz/sec
6 SNR 16.95 dB, 6002.301 MHz, -8.847 Hz/sec
7 SNR 16.48 dB, 6002.322 MHz, -8.392 Hz/sec
8 SNR 16.85 dB, 6002.342 MHz, -8.359 Hz/sec
9 SNR 16.90 dB, 6002.362 MHz, -7.839 Hz/sec
10 SNR 16.51 dB, 6002.382 MHz, -7.774 Hz/sec
11 SNR 16.63 dB, 6002.402 MHz, -7.058 Hz/sec
12 SNR 17.21 dB, 6002.423 MHz, -7.058 Hz/sec
13 SNR 17.25 dB, 6002.443 MHz, -7.058 Hz/sec
14 SNR 17.58 dB, 6002.463 MHz, -6.765 Hz/sec
15 SNR 17.46 dB, 6002.484 MHz, -6.245 Hz/sec
16 SNR 17.89 dB, 6002.504 MHz, -6.115 Hz/sec
17 SNR 18.06 dB, 6002.524 MHz, -6.050 Hz/sec
18 SNR 18.07 dB, 6002.544 MHz, -5.692 Hz/sec
19 SNR 17.87 dB, 6002.565 MHz, -5.594 Hz/sec
20 SNR 18.21 dB, 6002.585 MHz, -5.367 Hz/sec
21 SNR 18.58 dB, 6002.605

In [ ]:
import os

print('All Runs Done')

# Beep 2 times
for i_beep in range(2):
    try:
        # Beep in WSL
        if os.system("powershell.exe '[console]::beep(261.6,700)'") !=0:
            raise Exception('powershell.exe not found')
    except:
        # linux, probably doesn't work
        print('Beep!')
        os.system("echo -ne '\a'")